## 逻辑回归（正则化）算法实现
**目标:**
- 模块:
    - 参数初始化
    - 损失函数及梯度
    - 优化算法(如梯度下降) 
    - 输出预测概率
- 合并以上模块

### 关键步骤：“前向传播”&“反向传播

前向传播：
- 原始输入X
- 计算激活函数： $A = \sigma(w^T X + b) = (a^{(0)}, a^{(1)}, ..., a^{(m-1)}, a^{(m)})$
- 损失函数： $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})+\lambda\cdot\frac{1}{2}w^{T}w$

反向传播（梯度）：

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}+w\cdot\lambda$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [5]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
from sklearn.cross_validation import train_test_split

%matplotlib inline

## 模块一：参数初始化

In [8]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    w = np.zeros((dim, 1))
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

## 模块二：损失函数、梯度

In [9]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1 / (1 + np.exp(-z))    
    return s

In [80]:
def propagate(w, b, X, Y, lambd=0.0):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)
    lambd -- regularization parameter

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    # Cross-Entropy等于log-likelihood加个负号，前者求最小值后者求最大值。加入了L2-正则。
    A = sigmoid(np.dot(w.T, X) + b)             # compute activation
    cost = -np.sum( Y*np.log(A)+(1-Y)*np.log(1-A) ) / m - lambd * np.sum(w**2) / 2  # compute cost
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = np.dot(X, (A - Y).T) / m + w*lambd
    db = np.sum(A - Y) / m

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

## 模块三：优化算法--梯度下降

In [75]:
def optimize(w, b, X, Y, num_iterations, learning_rate, lambd=0.0, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    lambd -- regularization parameter
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        grads, cost = propagate(w, b, X, Y, lambd)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

## 模块四：输出预测概率

In [52]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

## 合并以上模块

In [76]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, lambd=0.0, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    lambd -- regularization parameter
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, lambd, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

## 测试

In [111]:
## 生成分类问题样本
X, y = make_classification(n_samples=10000, n_features=20, n_classes=2, 
                           n_informative=5, n_redundant=5, weights=[0.2,0.8], random_state=444)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=444)

## 调整为，一列一个样本
X_train_T = X_train.T
X_test_T = X_test.T
y_train_T = y_train.reshape((y_train.shape[0], 1)).T
y_test_T = y_test.reshape((y_test.shape[0], 1)).T

In [120]:
d = model(X_train_T, y_train_T, X_test_T, y_test_T, 
          num_iterations = 2000, learning_rate = 0.05, lambd = 0.05, print_cost = True)


Cost after iteration 0: 0.693147
Cost after iteration 100: 0.355626
Cost after iteration 200: 0.328307
Cost after iteration 300: 0.318930
Cost after iteration 400: 0.314979
Cost after iteration 500: 0.313089
Cost after iteration 600: 0.312101
Cost after iteration 700: 0.311549
Cost after iteration 800: 0.311222
Cost after iteration 900: 0.311021
Cost after iteration 1000: 0.310893
Cost after iteration 1100: 0.310809
Cost after iteration 1200: 0.310752
Cost after iteration 1300: 0.310714
Cost after iteration 1400: 0.310688
Cost after iteration 1500: 0.310669
Cost after iteration 1600: 0.310657
Cost after iteration 1700: 0.310648
Cost after iteration 1800: 0.310641
Cost after iteration 1900: 0.310637
train accuracy: 86.6268656716 %
test accuracy: 86.1515151515 %


## Bonus：相同数据集上，对比sklearn的逻辑回归API效果

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [113]:
lr = LogisticRegression(C=1.0)
lr.fit(X_train, y_train)

print('train accuracy: %.10f %%'% (accuracy_score(y_train, lr.predict(X_train))*100) )
print('test accuracy: %.10f %%'% (accuracy_score(y_test, lr.predict(X_test))*100) )

train accuracy: 86.7611940299 %
test accuracy: 86.0606060606 %
